In [48]:
import psycopg2
import pandas as pd


exposures = psycopg2.connect('service=exposure')
firecares = psycopg2.connect('service=firecares')

In [49]:
incidents = pd.read_sql_query('SELECT _id, incident_number, department_id FROM "Incidents"', exposures)
incidents = incidents.set_index('_id')
incidents

,incident_number,department_id
_id,,
5656200,WPB19010281,99102
8964672,9211981,88539
5656092,E1915246,90011
4756464,2037360,88539
5656103,191350082,91106
...,...,...
4754852,2039028,88539
4756199,2037661,88539
5656290,05152019-9174277,77989


In [50]:
exposures_df = pd.read_sql_query('SELECT _id, incident_id FROM "Exposures"', exposures)
exposures_count = exposures_df.groupby(['incident_id']).agg(['count'])
exposures_count.columns = exposures_count.columns.get_level_values(0)
exposures_count = exposures_count.rename(columns={'_id': 'count'})
exposures_count

,count
incident_id,
2679.0,4
2985.0,1
3055.0,2
3906.0,1
5479.0,1
...,...
20086759.0,1
20089347.0,1
20090494.0,1


In [86]:
fd = pd.read_sql_query('SELECT id, name, fdid, state FROM "firestation_firedepartment"', firecares)
fd = fd[fd.fdid != '']
fd['fdid_state'] = fd[['fdid', 'state']].agg('-'.join, axis=1)
fd = fd.set_index('fdid_state')
fd

,id,name,fdid,state
fdid_state,,,,
57050-LA,73102,7th Ward Volunteer Fire Department & Vermilion...,57050,LA
09214-GA,83383,Hahira Fire Department,09214,GA
71004-MS,73085,1st District Volunteer Fire Department/Colman ...,71004,MS
SH163-IL,82438,German Valley Fire Protection District,SH163,IL
25117-WV,95978,St. Joseph Volunteer Fire Department,25117,WV
...,...,...,...,...
08014-MI,78492,Columbia Township Fire Rescue,08014,MI
00303-MI,79857,Dorr Township Fire Department,00303,MI
01712-MI,79250,Dafter Township Fire Department,01712,MI


In [52]:
e = exposures_count
e

,count
incident_id,
2679.0,4
2985.0,1
3055.0,2
3906.0,1
5479.0,1
...,...
20086759.0,1
20089347.0,1
20090494.0,1


In [82]:
departments = pd.read_sql_query('SELECT _id, fd_id, state FROM "Departments"', exposures)
departments = departments.set_index('_id')
departments = departments[departments.fd_id != '']

departments['fdid_state'] = departments[['fd_id', 'state']].agg('-'.join, axis=1)

departments

#print(departments.to_csv())

,fd_id,state,fdid_state
_id,,,
76376,27555,CA,27555-CA
76291,00555,CA,00555-CA
76372,47557,CA,47557-CA
76373,23555,CA,23555-CA
76886,52555,CA,52555-CA
...,...,...,...
75500,25035,MA,25035-MA
77936,DD122,IL,DD122-IL
97477,11223,AZ,11223-AZ


In [54]:
inc_exp_count = incidents.join(e)
inc_exp_count

,incident_number,department_id,count
_id,,,
5656200,WPB19010281,99102,NaN
8964672,9211981,88539,NaN
5656092,E1915246,90011,NaN
4756464,2037360,88539,NaN
5656103,191350082,91106,NaN
...,...,...,...
4754852,2039028,88539,NaN
4756199,2037661,88539,NaN
5656290,05152019-9174277,77989,NaN


In [55]:
i = inc_exp_count.dropna()
i = i.set_index('department_id')
i

,incident_number,count
department_id,,
77863,20191015-0601412,1.0
77863,20200206-0070668,1.0
75500,F190031770,1.0
95671,19-0000627,1.0
77863,20200206-0070540,1.0
...,...,...
99082,2019WM0059816,1.0
99082,2019WM0022552,1.0
78503,F19166931,1.0


In [88]:
n = i.join(departments)
n = n.set_index('fdid_state')
n = n.drop(['state'], axis=1)
n

,incident_number,count,fd_id
fdid_state,,,
24000-MD,1121644,1.0,24000
24000-MD,1121294,1.0,24000
24000-MD,1096962,1.0,24000
24000-MD,1105765,1.0,24000
24000-MD,1105977,1.0,24000
...,...,...,...
05909-CO,2019WM0019905,1.0,05909
05909-CO,2019WM0022324,1.0,05909
05909-CO,2019WM0059623,1.0,05909


In [91]:
fn = n.join(fd)
fn = fn.drop(['fd_id'], axis=1)
fn = fn.rename(columns={'id': 'firecares_id'})
fn

,incident_number,count,firecares_id,name,fdid,state
fdid_state,,,,,,
00145-CO,ACFR20-0001920,2.0,90552,Adams County Fire Protection District,00145,CO
00145-CO,ACFR19-0007853,1.0,90552,Adams County Fire Protection District,00145,CO
00145-CO,ACFR19-0007559,2.0,90552,Adams County Fire Protection District,00145,CO
00145-CO,ACFR20-0002342,1.0,90552,Adams County Fire Protection District,00145,CO
00145-CO,ACFR20-0002349,2.0,90552,Adams County Fire Protection District,00145,CO
...,...,...,...,...,...,...
WG334-IL,20-0001197,2.0,92216,Plainfield Fire Protection District,WG334,IL
WG334-IL,20-0000003,1.0,92216,Plainfield Fire Protection District,WG334,IL
WG334-IL,20-0000066,1.0,92216,Plainfield Fire Protection District,WG334,IL


In [93]:
print(fn.to_csv(index=False))
fn.to_csv('./exposure_count_for_incident.csv', index=False)

incident_number,count,firecares_id,name,fdid,state
ACFR20-0001920,2.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0007853,1.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0007559,2.0,90552,Adams County Fire Protection District,00145,CO
ACFR20-0002342,1.0,90552,Adams County Fire Protection District,00145,CO
ACFR20-0002349,2.0,90552,Adams County Fire Protection District,00145,CO
ACFR20-0001582,1.0,90552,Adams County Fire Protection District,00145,CO
ACFR20-0002356,1.0,90552,Adams County Fire Protection District,00145,CO
ACFR20-0001614,1.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0003935,1.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0004099,2.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0005655,2.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0004078,2.0,90552,Adams County Fire Protection District,00145,CO
ACFR19-0008291,6.0,90552,Adams County Fire Protection District,00145,CO
ACFR20-000028